In [6]:
# This reload library is just used for developing the REPUBLIC hOCR parser 
# and can be removed once this module is stable.
%reload_ext autoreload
%autoreload 2


# This is needed to add the repo dir to the path so jupyter
# can load the republic modules directly from the notebooks
import os
import sys
repo_dir = os.path.split(os.getcwd())[0]
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

In [7]:

import republic.parser.republic_column_parser as column_parser
from elasticsearch import Elasticsearch
import republic.parser.republic_page_parser as page_parser
import republic.parser.republic_paragraph_parser as paragraph_parser
import republic.parser.republic_file_parser as file_parser

import copy



In [8]:
from republic.config.republic_config import base_config, set_config_inventory_num

inventory_num = 3767
data_dir = "/Users/marijnkoolen/Data/Projects/REPUBLIC/hocr"

inventory_config = set_config_inventory_num(base_config, inventory_num, data_dir)
print(inventory_config)
scan_files = file_parser.get_files(inventory_config["data_dir"])
print("Number of scan files:", len(scan_files))

scan_file = scan_files[10]
print(scan_file)


# scan num 751 is index start
# scan num 6 is resolution start

scan_file = scan_files[5]
scan_hocr = page_parser.get_scan_hocr(scan_file, inventory_config)
#print(scan_hocr["scan_type"])
#print(scan_hocr["page_box"])
if "double_page" in scan_hocr["scan_type"]:
    pages_hocr = page_parser.parse_double_page_scan(scan_hocr, inventory_config)
    for page_hocr in pages_hocr:
        print(page_hocr["page_num"], page_parser.get_page_type(page_hocr, debug=False))
        page_hocr["page_type"] += [page_parser.get_page_type(page_hocr, debug=False)]
        for column in page_hocr["columns"]:
            del column["lines"]
        #print(json.dumps(page_hocr, indent=2))

{'inventory_num': 3767, 'base_dir': '/Users/marijnkoolen/Data/Projects/REPUBLIC/hocr', 'page_index': 'republic_hocr_pages', 'page_doc_type': 'page', 'scan_index': 'republic_hocr_scans', 'scan_doc_type': 'scan', 'tiny_word_width': 15, 'avg_char_width': 20, 'remove_tiny_words': True, 'remove_line_numbers': False, 'normal_scan_width': 4840, 'word_conf_threshold': 10, 'column_gap_threshold': 50, 'gap_pixel_freq_ratio': 0.75, 'fulltext_char_threshold': 0.5, 'fulltext_words_threshold': 15, 'filter_words': ['|', '{', '$'], 'data_dir': '/Users/marijnkoolen/Data/Projects/REPUBLIC/hocr/3767/'}
Number of scan files: 774
{'scan_num': 11, 'inventory_num': 3767, 'inventory_year': 1712, 'inventory_period': ['1712-01-01', '1712-12-31'], 'filepath': '/Users/marijnkoolen/Data/Projects/REPUBLIC/hocr/3767/NL-HaNA_1.01.02_3767_0011.hocr'}
11 empty_page
12 resolution_page


In [11]:
import republic.parser.republic_page_parser as page_parser

column_config = {
    "avg_char_width": 20,
    "word_conf_threshold": 10,
    "column_gap_threshold": 50,
    "gap_pixel_freq_ratio": 0.75,
    "fulltext_char_threshold": 0.5,
    "fulltext_words_threshold": 15,
    "filter_words": ["|", "{", "$"]
}


scan_file = scan_files[196]
scan_hocr = page_parser.get_scan_hocr(scan_file, inventory_config)
print(scan_hocr["scan_num"], scan_hocr["scan_type"])
if "double_page" in scan_hocr["scan_type"]:
    pages_hocr = page_parser.parse_double_page_scan(scan_hocr, column_config)
    
# TO DO
# - determine page side and page number (scan * 2 for even scan * 2 - 1 for odd)

for page_hocr in pages_hocr:
    print("PAGE HOCR")
    #print(page_hocr)
    print(page_hocr["page_type"])
    columns_hocr = page_hocr["columns"]
    print("NUM COLUMNS:", len(columns_hocr))
    for column in columns_hocr:
        for line_index, line in enumerate(column["lines"]):
            print("column {}, line {} ({}, {}) ({}, {}):".format(line["column_num"], line_index+1, line["left"], line["right"], line["top"], line["bottom"]), 
                  " ".join([word["word_text"] for word in line["words"]]))

197 ['normal', 'double_page']
PAGE HOCR
['normal', 'double_column', 'resolution_page']
NUM COLUMNS: 2
column 1, line 1 (476, 1513) (341, 393): Den 1, April, ( 380 >
column 1, line 2 (478, 1357) (401, 449): digen Oorlogh, gier alleen Sr. David Mitchel,
column 1, line 3 (480, 1357) (464, 496): maer oock Sr. James Wishart op ordres van
column 1, line 4 (478, 1358) (504, 542): hare Majeltejt ‘naer Hollandt zyn gegaen, om
column 1, line 5 (477, 1361) (558, 583): met de Heeren Staten Generael, of der felver
column 1, line 6 (476, 1361) (605, 642): Gedeputeerdens te adjufteren de quote van ha-
column 1, line 7 (474, 1360) (659, 692): re Schepen van Oorlogh om in conjunêtie met
column 1, line 8 (475, 1362) (704, 735): die van hare Majelteyt te ageren, en het gecne
column 1, line 9 (474, 1362) (748, 791): volgens dele behandelingen was gedaen, isaen
column 1, line 10 (474, 1365) (796, 838): hare Majefteyts Principael-Sccrerariflen van Stact
column 1, line 11 (474, 1361) (852, 894): gefonden wae

In [13]:
column_config = {
    "avg_char_width": 20,
    "word_conf_threshold": 10,
    "column_gap_threshold": 40,
    "gap_pixel_freq_ratio": 0.75,
    "fulltext_char_threshold": 0.4,
    "fulltext_words_threshold": 15,
    "filter_words": ["|", "{", "$"]
}

inventory_num = 3234
inventory_config = set_config_inventory_num(base_config, inventory_num, data_dir)
print(inventory_config)
scan_files = file_parser.get_files(inventory_config["data_dir"])


for scan_file in scan_files[5:230]:
    scan_hocr = page_parser.get_scan_hocr(scan_file, inventory_config)
    if "double_page" in scan_hocr["scan_type"]:
        print("double page scan:", scan_hocr["scan_num"], scan_hocr["scan_type"])
        pages_hocr = column_parser.parse_double_page_scan(scan_hocr, column_config)
    else:
        print("NOT DOUBLE PAGE:", scan_hocr["scan_num"], scan_hocr["scan_type"])
        #continue
    print("num_lines:", len(scan_hocr["lines"]))
    for line in scan_hocr["lines"]:
        print(line["line_text"])
    #columns_hocr = split_lines_on_columns(dp_hocr, columns_starts)
    for page_hocr in pages_hocr:
        columns_hocr = page_hocr["columns"]
        if len(columns_hocr) != 2:
            print("page type:", page_hocr["page_type"], "\tnum columns:", len(columns_hocr))



{'inventory_num': 3234, 'base_dir': '/Users/marijnkoolen/Data/Projects/REPUBLIC/hocr', 'page_index': 'republic_hocr_pages', 'page_doc_type': 'page', 'scan_index': 'republic_hocr_scans', 'scan_doc_type': 'scan', 'tiny_word_width': 15, 'avg_char_width': 20, 'remove_tiny_words': True, 'remove_line_numbers': False, 'normal_scan_width': 4840, 'word_conf_threshold': 10, 'column_gap_threshold': 50, 'gap_pixel_freq_ratio': 0.75, 'fulltext_char_threshold': 0.5, 'fulltext_words_threshold': 15, 'filter_words': ['|', '{', '$'], 'data_dir': '/Users/marijnkoolen/Data/Projects/REPUBLIC/hocr/3234/'}
NOT DOUBLE PAGE: 6 ['special', 'extended']
num_lines: 110
ET Ei KT SW A ESE _
Gels, dt ve zoa Mich :
# A 4 Dem kh jeko Bou —_—
nn ja a 4 arts volts» {0 jeep U 1 # *
A} bs rr dp sjeiks Jee. ED Unk et cf vorh _ Es
=: 4 AU fj 4u “ut 9 GMI OP 7 ie j & De ==
{ TY 4e, ox v
w : 1 Nn ed ke Ooy 52 cs MD [de pi Hú NR ef Rans ee $
/; oeefo S es Got bebe, —n, Hi | 4 Á og De Ie OD od D vue 4 \
0 7 we ox Bode Loes edo /

NOT DOUBLE PAGE: 8 ['special', 'extended']
num_lines: 89
B” $ + : WN ow x\ — —— „.
/ voe vos er Ain SS) ce ou d,
_s “u te 6 JDedae mn
({ ne ecolo ot one A PE is en : Ed | ie —_ -
e oes a ) M- 10u Js 1 aussi he ( LEM an : * IE . pt ts dt ve AN v _——
| er oe eee ot a he se os RN „ nat Vous De 7 | ee ee oo as ne Be “
B epa jk Laak GI WE Ei 6 ©
Ee ID pt os | et ee ee are Dopke N
No web] fadore oJ vst Ue | Vg Tt Boke, (9 tT qicot À
‘oe An eN B ee ee tl ——an edelen MED KN 1 om
| gs Nd? vd Nn Co ST ae #í Wa Lu le? H (os 10 oep id Kr hmels, Kou q
ee vs (sto) Behte Ned Paesckas boh a ada ie
5 _— aud in XM dó eN TEN SRbelodae {L | De tt ig > ofte ks ed ie gede” Ui on )
Uu 5 CN Vion 4 Rn Oud of MEV mw | ie | | í & Gand ed PS da NN ne We Y os j
Eat oek ee 4 he Dd te , Dn IF Nn ge Je tet AEK 4 Coke Se ols
H _—_— aA hevel deel a =d | — ee n° (A4 6 “% - ks nn 7 opo “0 —
belegen hu ae D el Ke | ze w Kes N ij th [eers a
Fe B 10E: PN woe AdsR ! INN < Morrie, (Ds ied
es EN ee ER Le ee ee
|‘ ON he EN Abre

NOT DOUBLE PAGE: 12 ['special', 'extended']
num_lines: 57
i ariens ij eoegtens cle gig oat he Et ont oat Td 10 ®
3 a en SD SI GA, 1 Jas voorlegde Coa” Lt DR 7 u
Û Gies e: Ve" EE Lost Made ijk pepe Le 1
3 ° a if er lee lb dn Bk k Nen u cous
\ =
Se ‚ dae ed AAI AIS At & det Me 10 A60, (s 44 ei ) Dsg [e goh De AID TD Us |
Ei > Q (+ ó 0 \ Geatte fo Ahed, n_ Ì k ij
Ie Ae) voo Dd ln DS , Ds? bet om 5 ee doot HAD ID Wo
ì d ij Je? Dieemass TED AI in Va ete oger 6 De; x NV; Bonk ei 4
Ie ie kent ob He PSE Olne, Cid VOID SS ji A EE (ep ï
| ( fl # Gis ee Oude, DN wanen hj 1, Rr VS veneus, sol $ oes Pe ï #
C ikvan iN oud eee EE Ry Dd ow ie eo
Ad \ Í ) on mm DN (De orks Ds in > » ee Be 4 oP male wolle, cohn des EE
| IR € 5 bp wt vom Gelok FER EN aloud SEN (aoj Y 1 EN vis Hai, Dee
WL - pa Rana Re Hoi ANS Ie cad, SD a & _ ee sn " Sn Al
3 ens Va DS 0 Ades Tis ADA, Ad TOT das i e 4 AR tst & Otualas En De % Ml
Oos odt B Amo EN DN P ho Ne: Vol LD earn ge J
U / € SR bet Hole, ee ee a A NN )
7 DK dan lu SMS

NOT DOUBLE PAGE: 16 ['special', 'extended']
num_lines: 54
FF Lan Ne se Gode £ ee: > 6 Ls 7 / Gn 5, Wi tgeen (ND 4 4 A
| lewis Ea a kt, ek 9 md l De ® Ge Aloe DA Sd 6, BED IDA 4
| Drs bereidheid Wall BSD eef Si ae L JE Es ii % RE oi MScohids ode, |
LD geen eres haeen  ES ee
| er ee ii a
| eg Fe BER bets Zet, À =D oa A sd
| RD de Ú SPD dv SLD 5D work, FS de CT ies | > Nba TT, V/D : |
| goude tea! BE 29 yr oa 7 “5 E%) j : [it eee Pedi dk 4 LE dn CN foor 4 7’ MA
| Zouhe ho: Gt ON Beet EO A EW, sk ‘ h, 43041 MM (- et J orde oen Ge Se 9 el & ” se |
| DA ae GD Aude De no (rid Ù Ae | Y ” D. 9 mT a a 4
ee er kod; Nn pe 4 ee om odevmd, “ Yarns Ciola sa oee HE hs ll
/ My} ò 6 ed L jn ( : | ef 3 ade og Ang Ak ob er goon DL ei \ 7
| f Ees idee | ee ef ee
A, on of Dj NED PO Golda dd í | T TE * Jg u be 9 od AA) |
Ke, /4 9 : | 1 en Î £) ) ee _ M Vo &, a “Suus Det > TT |
! HY, / If Gog De fj DO dese ij on A Sr k A |
| ® L Ll /le G 0 SK (ee tid on odt von Bosk iN ì 5 e eg Ie. Us ee ome an Sm Lass |
VT N

NOT DOUBLE PAGE: 21 ['special', 'extended']
num_lines: 70
ee EN
1 A ons, eN EP / he > se afs Ep
5 id ® : Left
| ene en AN 7 Bijns en A Ide MP 94
| gig, oe green dee ie skea Ns | Ie 0D
| bes, toeg nt . odt d- oel DN ae Tw
CLE aoe GD tege, aak, óp Bessie Nen As ([s ic H
B ij Tiss 1 A er, OP ee sk l | Ns VNT f b Sá Áe 9 me B
ĳ A : AT ù, € Ee DE EN toute LL 3 Ie 9d eN SD Ak
Dn ha IF pp u Ko: Met ten Hat / Od ps, ulle | ij wo
if oke bn EO Me CH et Ts U > {_ | \ wije p Lia? vun, Sie ou
| (Dei a ld 4 0 rn á N 4 ie : . CS) wefhd U OD Pe de
9 een nt | SE ee
Ms Io iS eN ej 4! j Ie aw) et
ed, | IT wej hoads pie ĳ se eee pe O bv 1 d
welt ode ot Ve _— _ k Y ev a v A Ds
ip 9* NE U Ds CS A Ei ni x t Rige V eg Ad “==,
je (egg AED, Soames D= : y : Ne _ $ Pe ATA AN
Vv. — A 2 ee AND ON GD 9 ip . pom, De Oa beden
we ee MA en hs 1 oud, ns alte mtd b 14 0 JT LOST Ss bt bi
| egels oee Hi fati, vtt oale | OE
| De EEN 2, [Be kee a ‘ # > Ie vont geks is
ee Se ee ee B
\ Ee ada | EN EE
| Le ew D-4 HS en Cut fons 

...

NOT DOUBLE PAGE: 205 ['special', 'extended']
num_lines: 59
| 6 150 en
l fel fs ada)
ove) gea ef ptDe6 Ee
24 hp Gas CQ Gag» d ie -_
l Q 1 chen 5 Oue: EE + dwi IS ‘ "7
NN ast Uta Gadet ee oS{4y paps Ee Se Â Oe _
ae Be on ENE a Wou Seil as zaet ey
Cok 9. 423% aa? t ò3 ke g> Vak Jua Jr Ge Ue 96 Slk
fsf” eds 6 Bis IE 1649. 1# 3e os") 4 1 Wan, ae Arib
1 ce fan Ax Le PN wa Sed ro 2 3) 20
Je 9, m 43 j To: # > ä C et SI —
® aas) a 3230 ei a Ge ed Serre en 7
En, ar … | ff TOR 4 A :
Kn Ee GE DE, ND Me Ge
| Vege ad AS Ps <t SA IA (Gy js Goff nk ET eid ADs
EF Tass Ap a 5? Fens wa EG voor{: ee, di eat Vaes, ee
a ee ee ee
| eee SS JG pet vee) Ee Seep
k. a L 7 au PASTE CO od EN Ae HA ER
AI Jak ae se4. Jas Q JD D WE ® voo ED)
Ml ie mt Û too es eee ee mes”
ax ta AJ tot wj iP Dn f 8D 88 md hy (ses dst Agg !
ff ot fgQasD Neede Ne mee, Out oak sj d> Be +
[| le 5 ei) 3 EP fes wad > Ge DT ig Pus :
HL! ie D, 7 0 L ow Area) s Googr A, goo { {> are Qt PI
| gd: appi We Ce Tò% gsf tsj en Ì
| © Ee ee Ae #* ee AE I

NOT DOUBLE PAGE: 211 ['special', 'extended']
num_lines: 55
— _ ee - - ĳ ' abe NT trains aaa? |
a EE | Be Zg ee |
B CN eN 3 Des DD. 35382 t at fot, cu set Mias It li %4 AA \
! JP die Ee EO okredeg 2 | 4 Cg %, [ss ode icn, Le |
| Ee Pz ij Voor ie bats 95 aad Doper de voodb RG fe lj a Ee 1
’ rj + oe : Ne LA f Ee DS Wa N- 4, Ied Fa 4$ Baai > 5 YU
k avaÂ li ae Sr WI EE ee B ee ee Ie ee EE
| Cs Oef Gee [sns sekai Wren mijt Toy Jr hi +a + 5 eds ID voo 7e wan Ga 4 gJood 384
Petgat Orgy | 204 si Vaja] ee 50 ffe oe 2 SN |
Dier Me oe Ge Ee SG | ate “drsn A
H OQ es Ger Voeren Do Ue opir’ : Pie ee Gn Na IL 4Mesu
‘ vi ee ee, ee
i / 92049: 8 rade dad man 7 Toa fe Iu$ |
| DE EE NE dansend ens hade Aeteffda se |
| Zespecs ede ger ges fog | wepe nge sr eee |
| vies eee go) segers art {pd F6 goat, ee a EN
fi Kas eig and òf trof 7 Ge \me: Hefparidrnarid Au Ba ffastÀ 3 Gt 1
| Jody nd Ae Ly, 7 ge) Cc 9 Mast Pied > EN |
Ee ee ee pins Jet Pet TA a Nope |
j ded ee (24999 90 ds Dat aadig Voormasdg Vlad 59 oml d

NOT DOUBLE PAGE: 217 ['special', 'extended']
num_lines: 50
Sido IE Ee
tee Eese Tl ZEE, peen
’ Dm Gee RS EN |
[ Ee EE Tj Aa 7 s BS a SD 5u
le wees Bjng9fe7 Gat ig Aa ga VEN » oe? ots
| {99 8ed naval, ee eN geej) ee ogi{e7
‘ teng JE od ooo fijSo> da vrij eN Oe
K wp zoer dà w ej rn 34 en HI a Noor g+ 4
. EI EE er Nl
3 He [Ye a ee “Cady feed !
s EI oai Ee A Ee EO {
j do) fag A vnd Ee Giep Tm #
| Ke ee EG (2 AE ie 3 Jagt eben
[ verfen 3s> 28 Hip ate 074 ree Mer Heteren € CE
| Di gen M prs EE ar 908, MW side d #% Ee , eon) / 6%
9 el eer | Gerten Soteren f
: ee Re EW es TI OD
| feet ei ke EE f d efect Hjnherger, a VA
Í ecrsetsispey pfgeen fe | Bomert, Cera Marmar 9
E NN Issa ao 49 fe Ta, 3dsf. 3 (: De nete de Ee, nad ee b |
E wodt, Ee a N ie U den Van Hsllan Ht; l
EN > {034 sein hsl ease] en ee |
k CAAS en Y, a 19) j% „bergen TAN PTA) Ù K 4
| ie geta IVe, ee jj ie Pass Je Naar í ee Herle, Ilie ta 4
: ON er EG ee |
SA Aafke anche Se DLL ee: | Hsen, 4 SA Do ch Bint Al io ì
[oee Ee og este jede 

NOT DOUBLE PAGE: 224 ['special', 'extended']
num_lines: 52
l ee, Lsocf eefje Zane, id Ù y JA + Waah pees fj As: 9 ef og je 7
, Ee ee B Be et rond ed ‘ DE ee je tecoegpe  M
Â N ve ned, tdi otdounSn ho4 ds : je
| EE ee Be ae ese eraa et et U
U vroor a2f OE A4 ee [ADs re Goff off 1660 [cs ek tt l
| Pe ee ee
| ee eee DE  Vauise Vads en a se De ep: j
| Me Oor a bip eh Ce 8 Om 0, © Ge a a nen Î
[ ude id A fe (84 tot das V Vy OT Nan oe 08 fears A
| De a ag DA d> core a wiede sof { Ast fs Vantsd . Disdsr vou Caf |
| pd 9 ed Re) Va dan Gi pi ei 3. 43de 4, d
iK Do: eo fag se DT ASA a IN CD (250080 many DIe woog grz as |
1 x= #7} Ä > OS
| EE en |
{ 1 (ma geen aah, Sa a > loor X% 44 Oe Ds A iN
aas A Voors pster af gs 497 Pe. Sepepe te Las: “ol erts 0 fg
{| dsg 8 Gj AQ ” + 9 ra er NAD IMO
Ee, Be 6 EN ap wesdl Gra GK se Van fijn, Sus 224 DES Marry 16E |
iN Gd IE ee JMA {0 vos 50 SBA SE 519 Go. br 62. 63. 64 Gs
ee 6 ee Fe 7 Quo: 2» g99-% eso (ded) vlot giet oo RN svet sud op : |
| RE ee ned GS plan s

NOT DOUBLE PAGE: 230 ['special', 'extended']
num_lines: 55
' Ee (4 6 eN ee
/ ITI IJ vod. =- 7 220
Koss C, on Din qe U sijt a 0é and. 9 Re pee a 4 — «2337 VO
’. Axa Ga fand: SN ee Wa et Kn A as : Ne Sa osfse0 98 go)
9 Sup lois st Is-4.8a Ds. ey Deaf eg as fo mines, O/T, Zat ga lo S
Is AN P À Is n “opel Veen “oSud g Ies 5 .s9{ S
AS LosòJp iss 4 Wa Was pO A Dot ae Aal Vary Gi045 el ei |
ede U 5 4 d iof sars
BE EE Raa Ads ij vet { » 8, 19 NE
| agr qa Iet ijl Stond NV DSL sad 2 zen VE Se ee ® Aad f
ee pee Gd, |
| Ooernuan maa Dag seat g5des alÀ et ges Ne |
HN Se el (53 SNG Qty eee res ds |
e j J 11 IA Mi Se geste Sj ee ed 94u port east Den |
| Ne tad Pattand, ea 139 | e 4 3 (CG bod 7 dee sn Jee Ls {> !
| ei IS Sehraoindiman ik es Ototiauut Voor 3ral23, CDs, fes] 63 j
| > re SEP es CC NN, tage DP cd Eg oant, ge |
9 Asc fpsns &, m4 sin co 4 “+ Cent (tt Varo fn ge
NE ij Vv d eet. \
| d. es ae ee: iebaret jj Gens 4 oe Dg in {kome
‘( ud IerG Sit ge Sijt (io Or Nn) oee Hoi ff, Tot Sera ias, Î
Sos

In [17]:
import republic.elastic.republic_elasticsearch as rep_es

es = Elasticsearch()

#rep_es.delete_es_index(base_config["page_index"])
#rep_es.delete_es_index(base_config["scan_index"])

inventory_numbers = [
    3767,
    #3768,
    #3769,
    #3783,
    #3796,
    #3798,
    #3853
]

for inventory_num in inventory_numbers:
    rep_es.parse_inventory(es, inventory_num, base_config, data_dir)



{'inventory_num': 3767, 'base_dir': '/Users/marijnkoolen/Data/Projects/REPUBLIC/hocr', 'page_index': 'republic_hocr_pages', 'page_doc_type': 'page', 'scan_index': 'republic_hocr_scans', 'scan_doc_type': 'scan', 'tiny_word_width': 15, 'avg_char_width': 20, 'remove_tiny_words': True, 'remove_line_numbers': False, 'normal_scan_width': 4840, 'word_conf_threshold': 10, 'column_gap_threshold': 50, 'gap_pixel_freq_ratio': 0.75, 'fulltext_char_threshold': 0.5, 'fulltext_words_threshold': 15, 'filter_words': ['|', '{', '$'], 'data_dir': '/Users/marijnkoolen/Data/Projects/REPUBLIC/hocr/3767/'}
NOT DOUBLE PAGE: 1 ['special', 'small']
double page scan: 2 ['normal', 'double_page']
double page scan: 3 ['normal', 'double_page']
double page scan: 4 ['normal', 'double_page']
double page scan: 5 ['normal', 'double_page']
double page scan: 6 ['normal', 'double_page']
double page scan: 7 ['normal', 'double_page']
double page scan: 8 ['normal', 'double_page']
double page scan: 9 ['normal', 'double_page']
d

double page scan: 162 ['normal', 'double_page']
double page scan: 163 ['normal', 'double_page']
double page scan: 164 ['normal', 'double_page']
double page scan: 165 ['normal', 'double_page']
double page scan: 166 ['normal', 'double_page']
double page scan: 167 ['normal', 'double_page']
double page scan: 168 ['normal', 'double_page']
double page scan: 169 ['normal', 'double_page']
double page scan: 170 ['normal', 'double_page']
double page scan: 171 ['normal', 'double_page']
double page scan: 172 ['normal', 'double_page']
double page scan: 173 ['normal', 'double_page']
double page scan: 174 ['normal', 'double_page']
double page scan: 175 ['normal', 'double_page']
double page scan: 176 ['normal', 'double_page']
double page scan: 177 ['normal', 'double_page']
double page scan: 178 ['normal', 'double_page']
double page scan: 179 ['normal', 'double_page']
double page scan: 180 ['normal', 'double_page']
double page scan: 181 ['normal', 'double_page']
double page scan: 182 ['normal', 'double

double page scan: 334 ['normal', 'double_page']
double page scan: 335 ['normal', 'double_page']
double page scan: 336 ['normal', 'double_page']
double page scan: 337 ['normal', 'double_page']
double page scan: 338 ['normal', 'double_page']
double page scan: 339 ['normal', 'double_page']
double page scan: 340 ['normal', 'double_page']
double page scan: 341 ['normal', 'double_page']
double page scan: 342 ['normal', 'double_page']
double page scan: 343 ['normal', 'double_page']
double page scan: 344 ['normal', 'double_page']
double page scan: 345 ['normal', 'double_page']
double page scan: 346 ['normal', 'double_page']
double page scan: 347 ['normal', 'double_page']
double page scan: 348 ['normal', 'double_page']
double page scan: 349 ['normal', 'double_page']
double page scan: 350 ['normal', 'double_page']
double page scan: 351 ['normal', 'double_page']
double page scan: 352 ['normal', 'double_page']
double page scan: 353 ['normal', 'double_page']
double page scan: 354 ['normal', 'double

double page scan: 505 ['normal', 'double_page']
double page scan: 506 ['normal', 'double_page']
double page scan: 507 ['normal', 'double_page']
double page scan: 508 ['normal', 'double_page']
double page scan: 509 ['normal', 'double_page']
double page scan: 510 ['normal', 'double_page']
double page scan: 511 ['normal', 'double_page']
double page scan: 512 ['normal', 'double_page']
double page scan: 513 ['normal', 'double_page']
double page scan: 514 ['normal', 'double_page']
double page scan: 515 ['normal', 'double_page']
double page scan: 516 ['normal', 'double_page']
double page scan: 517 ['normal', 'double_page']
double page scan: 518 ['normal', 'double_page']
double page scan: 519 ['normal', 'double_page']
double page scan: 520 ['normal', 'double_page']
double page scan: 521 ['normal', 'double_page']
double page scan: 522 ['normal', 'double_page']
double page scan: 523 ['normal', 'double_page']
double page scan: 524 ['normal', 'double_page']
double page scan: 525 ['normal', 'double

double page scan: 676 ['normal', 'double_page']
double page scan: 677 ['normal', 'double_page']
double page scan: 678 ['normal', 'double_page']
double page scan: 679 ['normal', 'double_page']
double page scan: 680 ['normal', 'double_page']
double page scan: 681 ['normal', 'double_page']
double page scan: 682 ['normal', 'double_page']
double page scan: 683 ['normal', 'double_page']
double page scan: 684 ['normal', 'double_page']
double page scan: 685 ['normal', 'double_page']
double page scan: 686 ['normal', 'double_page']
double page scan: 687 ['normal', 'double_page']
double page scan: 688 ['normal', 'double_page']
double page scan: 689 ['normal', 'double_page']
double page scan: 690 ['normal', 'double_page']
double page scan: 691 ['normal', 'double_page']
double page scan: 692 ['normal', 'double_page']
double page scan: 693 ['normal', 'double_page']
double page scan: 694 ['normal', 'double_page']
double page scan: 695 ['normal', 'double_page']
double page scan: 696 ['normal', 'double

In [462]:
import republic.parser.republic_index_page_parser as index_parser

def make_page_type_query(inventory_num: int, page_type: str, size: int = 10000) -> object:
    return {
        "query": {
            "bool":{
                "must": [
                    {"match": {"inventory_num": inventory_num}},
                    {"match": {"page_type": page_type}}
                ]
            }
        },
        "size": size
    }

inventory_num = 3767
page_type = "index_page"
#page_type = "resolution_page"

query = make_page_type_query(inventory_num, page_type)

response = es.search(index=base_config["page_index"], doc_type=base_config["page_doc_type"], body=query)
print(response['hits']['total'])

pages = [hit["_source"]["page_num"] for hit in response['hits']['hits']]
print(pages)


#for hit in response['hits']['hits']:
#    page_hocr = rep_es.parse_es_page_doc(hit["_source"])
#    print(index_parser.count_number_chars(page_hocr))


385
[847, 872, 877, 868, 897, 882, 960, 953, 942, 928, 900, 755, 756, 741, 744, 968, 1023, 1017, 981, 1088, 1103, 1230, 1232, 1235, 1251, 1252, 1237, 1240, 1264, 1316, 1377, 1384, 20, 43, 46, 111, 112, 104, 107, 181, 242, 268, 273, 260, 367, 371, 418, 411, 390, 412, 394, 538, 535, 508, 582, 569, 686, 720, 1520, 1523, 1509, 1510, 1502, 1516, 1540, 87, 17, 92, 108, 221, 222, 265, 266, 372, 386, 417, 396, 497, 495, 496, 506, 507, 484, 456, 533, 534, 558, 536, 510, 572, 634, 679, 665, 740, 718, 1385, 1386, 1380, 1508, 1528, 1529, 1530, 1518, 1521, 1514, 1536, 786, 788, 776, 1067, 1013, 1150, 1144, 1269, 1272, 1262, 1305, 918, 920, 914, 878, 896, 922, 883, 844, 939, 933, 982, 1071, 1072, 1099, 1236, 1214, 1248, 1258, 1330, 47, 53, 91, 137, 203, 204, 241, 186, 245, 168, 178, 274, 259, 262, 360, 373, 481, 483, 461, 462, 468, 453, 457, 503, 504, 517, 518, 509, 491, 513, 547, 579, 571, 646, 689, 690, 696, 693, 833, 770, 775, 881, 886, 865, 866, 871, 874, 898, 967, 948, 934, 921, 1100, 1246, 124

In [482]:
pages_hocr = [rep_es.parse_es_page_doc(hit["_source"]) for hit in response['hits']['hits']]

for page_hocr in pages_hocr:
    if page_hocr["page_num"] < 1500:
        page_type = page_parser.get_page_type(page_hocr, debug=False)
        print(page_hocr["page_num"], page_type, page_hocr["num_words"])

847 index_page 313
872 resolution_page 1046
877 resolution_page 923
868 resolution_page 868
897 unknown_page_type 891
882 resolution_page 839
960 resolution_page 1098
953 resolution_page 847
942 unknown_page_type 611
928 resolution_page 1072
900 resolution_page 990
755 index_page 991
756 resolution_page 1186
741 resolution_page 922
744 resolution_page 1122
968 resolution_page 1116
1023 resolution_page 1013
1017 resolution_page 861
981 unknown_page_type 720
1088 resolution_page 1121
1103 resolution_page 952
1230 resolution_page 899
1232 resolution_page 870
1235 resolution_page 964
1251 resolution_page 1056
1252 resolution_page 1047
1237 unknown_page_type 655
1240 resolution_page 927
1264 resolution_page 976
1316 resolution_page 1042
1377 unknown_page_type 574
1384 resolution_page 885
20 resolution_page 916
43 unknown_page_type 607
46 resolution_page 1086
111 resolution_page 870
112 resolution_page 747
104 resolution_page 825
107 index_page 662
181 resolution_page 978
242 resolution_page

949 resolution_page 919
1007 resolution_page 1082
1014 resolution_page 880
983 resolution_page 818
997 resolution_page 791
1022 unknown_page_type 674
1263 resolution_page 969
1271 resolution_page 791
1260 unknown_page_type 654
